In [1]:
import cv2
import numpy as np

In [2]:
img_np = np.ones([5, 5])

print(img_np)

[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


In [3]:
kernel = np.array([-1, 0, 1])

print(kernel)

[-1  0  1]


In [4]:
# 디지털 신호처리의 FIR필터를 공부해보면 좋음

# cv는 OpenCV라는 C++로 작성된 라이브러리를 python으로 가져다 사용하기 때문에 python의 행렬 정보를 OpenCV용으로 변환할 필요가 있음
# cv2.resize 목적: 타입 컨버팅
img_cv = cv2.resize(img_np, (5, 5))

print(img_cv)

[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


In [5]:
kernel = cv2.resize(kernel, (1, 3))

print(kernel)

[[-1]
 [ 0]
 [ 1]]


In [6]:
img_cv[0][0] = 0
img_cv[2][2] = 2
img_cv[4][4] = 3

print(img_cv)

[[0. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 2. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 3.]]


In [7]:
# cv2.fliter2D 컨벌루션이 라플라스 변환과 푸리에 변환이랑 관련이 있다.
# 푸리에 변환(주파수 영역으로 변환)이 완료된 두 함수의 곱은 시간 영역에서 컨벌루션과 동일하다.
# (증명: 자코비안 행렬과 라플라스 변환이 필요함)
# 위의 내용은 FIR필터의 가장 기본 근간이 됨
#                                앵커
conv_test = cv2.filter2D(img_cv, -1, kernel)

print(conv_test)

[[ 0.  0.  0.  0.  0.]
 [ 1.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  2.]
 [ 0.  0.  0.  0.  0.]]


In [8]:
import copy

# copy.deepcopy() 하는 이유
# C++관점에서 포인터를 공유하는 상황이 될 수 있어 malloc을 통해 해결하는 것과 같다.
# 자바관점에서 객체를 공유하는 상황이 될 수 있어 new를 통해 해결하는 것과 같다.
def filter_2d(input_img, kernel):
    input_img = copy.deepcopy(input_img)
    output_img = copy.deepcopy(input_img)
    # 상용화시 이렇게 딥카피를 두번하는 건 하면 안되지만 이미지를 보려고 하는 것이기 때문에 그냥 함
    
    print('input_img.shape[0]:', input_img.shape[0])
    print('input_img.shape[1]:', input_img.shape[1])
    
    # 연산을 올바르게 하고 있는가 확인 필요
    # 사실 5개정도는 for문보다는 바로 확인이 좋다. 파이프라인이 깨지기 때문
    # for i in range(input_img.shape[0]):
    #    print(input_img[i])
        
    # 컨벌루션 연산이 적용되는 대상은 이미지이다.
    # 이미지 -> (다차원)행렬
    for row in range(input_img.shape[0]):
        
        output_img[row][0] = input_img[row][1] * kernel[0] + \
                        input_img[row][0] * kernel[1] + \
                        input_img[row][1] * kernel[2]
        
        for col in range(1, input_img.shape[1] -1):
            output_img[row][col] = input_img[row][col -1] + kernel[0] +\
                                input_img[row][col] * kernel[1] + \
                                input_img[row][col + 1] * kernel[2]
            
        output_img[row][input_img.shape[1] - 1] = \
            input_img[row][input_img.shape[1] - 2] * kernel[0] + \
            input_img[row][input_img.shape[1] - 1] * kernel[1] + \
            input_img[row][input_img.shape[1] - 2] * kernel[2]
            
    return output_img.T

In [9]:
conv_img = filter_2d(img_cv, kernel)
conv_img2 = cv2.filter2D(img_cv, -1, kernel)

print(conv_img)
print(conv_img == conv_img2)

input_img.shape[0]: 5
input_img.shape[1]: 5
[[0. 0. 0. 0. 0.]
 [0. 1. 2. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 2. 1. 3.]
 [0. 0. 0. 0. 0.]]
[[ True  True  True  True  True]
 [False False False False False]
 [False False False False False]
 [False False False False False]
 [ True  True  True  True  True]]


In [10]:
# Microsoft의 FPGA: 하드웨어를 소프트웨어로 변형 가능. 삼성 메모리 반도체에도 이용됨.

In [11]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

In [12]:
classifier = VGG16()

In [13]:
classifier.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [17]:
suitImg = image.load_img("suit.jpg", target_size=(224, 224))

In [18]:
transformed_img = image.img_to_array(suitImg)
print(transformed_img.shape)

(224, 224, 3)


In [19]:
transformed_img = np.expand_dims(transformed_image, axis = 0)
print(transformed_img.shape)
print(transformed_img)

NameError: name 'transformed_image' is not defined

In [ ]:
transformed_img = preprocess_input(transformed_img)
print(transformed_img.shape)
print(transformed_img)

In [ ]:
pred = classifier.predict(transformed_img)
print(pred)

In [ ]:
print(pred.shape)

In [ ]:
label = decode_predictions(pred)

In [ ]:
print('%s (%.2f%%)' % (label[0][0][1], label[0][0][2] * 100))

In [ ]:
print('%s (%.2f%%)' % (label[0][1][1], label[0][1][2] * 100))

In [ ]:
print('%s (%.2f%%)' % (label[0][2][1], label[0][2][2] * 100))

In [ ]:
print('top 5!')

for i in range(5):
    print('%16s (%.2f%%)' % (label[0][i][1], label[0][i][2] * 100))